Classification of five different kinds of flowers by their images.

The data set contains partitions of images of different flowers. These are {daisy, dadelion, rose, sunflower, tulip}.

In [2]:
# Preliminaries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models

import os, shutil
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.simplefilter(action='ignore')


import keras_preprocessing
from keras_preprocessing import image

from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical

from PIL import Image # image module is a submodule of PIL that contains the classes and functions for working with individual images

from keras_preprocessing.image import img_to_array
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.preprocessing import LabelEncoder


In [ ]:
# Defining the classes/categories as per dataset
categories=['daisy', 'rose', 'tulip', 'dandelion', 'sunflower']
directory = 'Flowers'

In [ ]:
os.listdir(directory)

In [ ]:
# Number of images
images_count = []
for category in os.listdir(directory):
    images_count.append(len(os.listdir(r"" + directory + "/" + category)))
images_count

In [ ]:
data = []
labels = []

for category in os.listdir(directory):
    for images in os.listdir(os.path.join(directory, category))[:733]:
        image = Image.open(os.path.join(directory, category, images))
        image = image.resize((150, 150))
        image_array = img_to_array(image) / 255
        data.append(image_array)
        labels.append(category)


In [ ]:
# One hot encoding of labels

data = np.array(data)
labels = np.array(labels)

le = LabelEncoder()
labels = le.fit_transform(labels)
labels = keras.utils.to_categorical(labels, len(np.unique(labels)))


In [ ]:
# Split the data into train and validation/test
X_train, X_tval, y_train, y_tval = train_test_split(data, labels, stratify = labels, test_size=0.3)

In [ ]:
# Split the validation/test into validation and test data (50% of 30% = val and test each are 15% of the total dataset)
X_val, X_test, y_val, y_test = train_test_split(X_tval, y_tval, stratify = y_tval, test_size=0.5)

In [ ]:
# Data augmentation on train data
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

X_train_augmented, y_train_augmented = datagen.flow(X_train, y_train, shuffle = True, batch_size=len(X_train)).next()

X_train = np.concatenate((X_train, X_train_augmented))
y_train = np.concatenate((y_train, y_train_augmented))


In [ ]:
# After adding the augmented images to the train dataset -> shuffle again

In [ ]:
indices = np.random.permutation(X_train.shape[0])
X_train = X_train[indices]
y_train = y_train[indices]

In [ ]:
def get_model():
    model = keras.Sequential()

    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
    model.add(keras.layers.MaxPooling2D((2, 2)))

    model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2)))

    model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2)))

    model.add(keras.layers.Flatten())

    model.add(keras.layers.Dense(512, activation='relu', 
    kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)))
    model.add(keras.layers.Dropout(0.5))

    model.add(keras.layers.Dense(5, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:
df = pd.DataFrame()

batch_sizes = [10, 16, 32, 64, 128]
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
callbacks = [early_stop]

for batch_size in batch_sizes:
    model = get_model()
    history = model.fit(
            X_train, y_train,
            epochs=100,
            validation_data=(X_val, y_val),
            batch_size = batch_size,
            callbacks = callbacks)
    val_acc = history.history["val_accuracy"][-1]
    df = df.append({'batch size' : batch_size,
                    'validation accuracy': val_acc}, ignore_index=True)


In [ ]:
df

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, batch_size=batch_size)
testplan = df.append({'batch size' : batch_size,
                'validation accuracy': val_acc,
                'test accuracy': test_acc,
                'test loss': test_loss }, ignore_index=True)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
from keras.models import load_model

model = load_model('path/to/saved/model.h5')

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

In [ ]:
results = model.evaluate(test_images, test_labels)
results